In [3]:
# import packages
import json
import MeCab
import demoji
import re
from stop_words import stop_words
import gensim, logging

ImportError: dlopen(/usr/local/Caskroom/miniforge/base/envs/thesis/lib/python3.12/site-packages/MeCab/_MeCab.cpython-312-darwin.so, 0x0002): tried: '/usr/local/Caskroom/miniforge/base/envs/thesis/lib/python3.12/site-packages/MeCab/_MeCab.cpython-312-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64')), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/Caskroom/miniforge/base/envs/thesis/lib/python3.12/site-packages/MeCab/_MeCab.cpython-312-darwin.so' (no such file), '/usr/local/Caskroom/miniforge/base/envs/thesis/lib/python3.12/site-packages/MeCab/_MeCab.cpython-312-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64'))

In [24]:
# !pip3 download mecab-python3
# !cd /Users/taylory/Desktop/*Thesis/thesis/
# !tar xfv mecab-python3-1.0.5.tar.gz
# !cd mecab-python3-1.0.5    
# !brew install mecab
# !python3 setup.py build
# !python3 setup.py install
# !pip3 list


Solving environment: unsuccessful initial attempt using frozen solve. Retrying with flexible solve.
Solving environment: unsuccessful initial attempt using frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - mecab

Current channels:

  - https://conda.anaconda.org/conda-forge/osx-arm64
  - https://conda.anaconda.org/conda-forge/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.



Note: you may need to restart the kernel to use updated packages.


In [16]:
!pip3 uninstall mecab-python3

In [2]:
!install_name_tool -change "/usr/lib/libmecab.dylib" /usr/local/lib/libmecab.dylib /usr/local/Caskroom/miniforge/base/envs/thesis/lib/python3.12/site-packages/MeCab/_MeCab.cpython-312-darwin.so

/usr/local/bin/MeCab


In [ ]:
ARCHFLAGS="-arch arm64" pip3 install mecab  --compile --no-cache-dir

